# Simulation of Physical Cavity Setup

![](images/cavity.png)

* the length of the `cavityspace` is dependent on the focal length of the mirrors
    * in our case the focal length is 10cm so the `cavityspace` will be 20cm
* whether the cavity is in resonance depends on the the exact distance between the two mirrors
    * if it's a multiple of the `laser`'s frequency the cavity will be in resonance
    * in Finesse2 this is represented by the **tuning** (phi) of a mirror

---

Small distance adjustments (nanometers) should be accomplished via changing the tuning of a component, large adjustments via the length parameter of a space.
**Small adjustments will NOT work via the length parameter!**

## High Reflectivity

* `piezomirror` R of 99,7%
* `lambda` for phi is set in [kat.ini](kat.ini) (in our case: `lambda 6.33E-7`, 633 nanometers)
* `piezomirror`: a positive phi moves the mirror `node4` closer to `node3`

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
# LASER <Node0> PRESPACE <Node1>[ITM]<Node2> CAVITYSPACE <Node3> [ETM] <Node4> DIODE 
# for mirror1: 
# * a mirror to represent the convex AR surface
# * a space for the thickness of the mirror, with a refractive index;
# * a mirror representing the concave HR surface 

# https://www.thorlabs.de/drawings/d929f0a3f9a9311d-4A3FC8A0-C769-CB45-55D1BB5AD81654C0/CM127P-012-E02-AutoCADPDF.pdf
# our mirror 

# control components from here
# TODO: enter correct parameters
# inspired by https://git.ligo.org/finesse/pykat/-/blob/master/examples/cavity_scan.py
# see also 02-useful.ipynb

parameters = {
    "l": { #laser
        "p": 10000000,  # input laser power [W]
        "fo": 0,  # frequency offset
        "ph": 0,  # phase
    },
    "c":{ #cavity
        "len": 100, #length of cavity
            'itm': {				# ITM input test mass (our fixed mirror1 )
                'radius': 0.0127,		# [m] r = 1/2 inch = 0.127m(
                'radius_of_curvature': {
                    'inner': {		# inner concave surface of mirror - highly reflective
                        'x': -5.7,	# [m] TODO
                        'y': -5.7	# [m] TODO
                    },
                    'outer': {		# outer surface of mirror - anti reflective
                        'x': 0,	# [m] TODO
                        'y': 0	# [m] TODO
                    }
                },
                'thickness': 0.027,		# [m]
                'reflectivity': {		# power reflectivity
                    'inner': 0.997,		# inner refl. of concave surface
                    'outer': 0.001		# outer refl. of convex surface
                },
                'transmission': {		# power transmission ??
                    'inner': 0.005,
                    'outer': 0.999
                },
                'tuning_angle': {		# phi
                    'inner': 0,
                    'outer': 0
                },
                'misalignment': {		# mirror misalignment [rad]
                    'inner': {
                        'x': 0,
                        'y': 0
                    },
                    'outer': {
                        'x': 0,
                        'y': 0
                    }
                }
            },
                'etm': {				# ETM end test mass (our piezo mirror )
                'radius': 0.023225,		# [m]
                'radius_of_curvature': {
                    'inner': {		# inner concave surface of mirror
                        'x': 5.7,	# [m]
                        'y': 5.7	# [m]
                    },
                    'outer': {		# outer convex surface of mirror
                        'x': 1.7763,	# [m]
                        'y': 1.7763	# [m]
                    }
                },
                'thickness': 0.027,		# [m]
                'reflectivity': {		# power reflectivity
                    'inner': 0.997,		# inner refl. of concave surface
                    'outer': 0.001		# outer refl. of convex surface
                },
                'transmission': {		# power transmission ??
                    'inner': 0.005,
                    'outer': 0.999
                },
                'tuning_angle': {		# phi
                    'inner': 0,
                    'outer': 0
                },
                'misalignment': {		# mirror misalignment [rad]
                    'inner': {
                        'x': 0,
                        'y': 0
                    },
                    'outer': {
                        'x': 0,
                        'y': 0
                    }
                }
            }
}


# add a laser
kat.add(
    pykat.components.laser(
        "laser",  # name
        "node0",  # node 0
        parameters["l"]["p"],  # laser power
        parameters["l"]["fo"],  # laser frequency_offset
        parameters["l"]["ph"],  # laser phase
    )
)

# add 10 cm prespace
kat.add(
    pykat.components.space(
        "prespace",  # name
        "node0",  # node 0
        "node1",  # node 1
        0.1       # length [m]
    )
)

# add cavity component
# ...no finished
    
reflectivity = 0.997
code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

## Low Reflectivity


* `piezomirror` R of 90%
* a lower reflectivity reduces the power output of the cavity

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *
import matplotlib.pyplot as plt

kat = finesse.kat()
reflectivity = 0.9

code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

In [ ]:
from IPython.display import display
import ipywidgets as widgets

switch = True

button = widgets.Button(description="toggle scaling")
output = widgets.Output()

display(button, output)


def on_button_clicked(b):
    global switch
    if switch:
        plt.ylim(top=1259999.864594087)
    else:
        plt.ylim(top=135888.67754189012)
    switch = not switch


button.on_click(on_button_clicked)

## Reflectivity with phi == 0

When **phi** is zero degress the cavity is in perfect resonance.

* phi is fixed to zero
* the reflectivity of `piezomirror` is swept from 90% to 100%
* 1.000 steps
* higher reflectivity leads to exponentially more cavity power output

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis piezomirror R lin .9 1 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)
print(kat.laser.P)

## Laser power at phi==0

* phi is fixed to zero
* the power output of `laser` is swept from 0 to 3 Megawatts
* higher power input leads to a linear increase of cavity power output
* 1.000 steps

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis laser P lin 0 3M 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)
print(kat.laser.P)